<a href="https://colab.research.google.com/github/ana-fav/Portifolio/blob/main/Criando_ABT_com_safras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Importanto bibliotecas
import os
import numpy as np
import pandas as pd
import dateutil

# 1. Manipulação da Base de Modelagem

## 1.1 Carregando os Dados

In [2]:
# Diretórios para o carregamento do Dataset
pasta_raiz = '/content/drive/MyDrive/Portifólio Data Science/datasets'
DATA_DIR = os.path.join(pasta_raiz, 'olist')

### 1.1.1 Pedidos
Esta é a tabela fato, em que cada registro representa um pedido realizado

In [3]:
df_orders = pd.read_csv(os.path.join(DATA_DIR, 'olist_orders_dataset.csv'), parse_dates=['order_approved_at'])
df_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [4]:
# Verificando a quantidade total de pedidos, clientes e registros na base
print(f'Qtde. Pedidos: {df_orders["order_id"].nunique()}')
print(f'Qtde. Clientes: {df_orders["customer_id"].nunique()}')
print(f'Qtde. registros: {df_orders.shape[0]}')

Qtde. Pedidos: 99441
Qtde. Clientes: 99441
Qtde. registros: 99441


In [5]:
# Verificando o intervalo de datas na base
df_orders.agg({'order_approved_at': ['min', 'max']})

,order_approved_at
min,2016-09-15 12:16:38
max,2018-09-03 17:40:06


In [6]:
# Explorando os status das vendas
df_orders['order_status'].value_counts()

delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: order_status, dtype: int64

In [7]:
# Explorando os status das vendas de forma normalizada
df_orders['order_status'].value_counts(normalize=True)

delivered      0.970203
shipped        0.011132
canceled       0.006285
unavailable    0.006124
invoiced       0.003158
processing     0.003027
created        0.000050
approved       0.000020
Name: order_status, dtype: float64

Temos uma base com dados de 24 meses, com um total de 99.441 vendas realizadas, sendo que cada cliente comprou apenas uma vez neste período de histórico.

A base conta com 8 diferentes status de pedidos, sendo que 97% dos pedidos possui o status "delivered"

### 1.1.2 Transações

Nesta tabela cada registro irá representar um item de um pedido

In [8]:
df_order_items = pd.read_csv(os.path.join(DATA_DIR, 'olist_order_items_dataset.csv'))
df_order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [9]:
# Verificando a quantidade total de produtos e vendedores
print(f'Qtde. Pedidos: {df_order_items["order_id"].nunique()}')
print(f'Qtde. Produtos: {df_order_items["product_id"].nunique()}')
print(f'Qtde. Vendedores: {df_order_items["seller_id"].nunique()}')

Qtde. Pedidos: 98666
Qtde. Produtos: 32951
Qtde. Vendedores: 3095


In [10]:
# Verificando o intervalo de na preços dos protudos e dos fretes
df_order_items[['price', 'freight_value', ]].agg(['min', 'max'])

,price,freight_value
min,0.85,0.00
max,6735.00,409.68


Esta tabela apresenta os dados de 98.666 vendas em que 32951 produtos foram vendidos por preços que variam de 0,85 até 6.735,00, e fretes que variam entre gratuitos e 409,68.

As vendas foram conduzidas por 3.095 vendedores


### 1.1.3. Vendedores

Nesta tabela cada registro representa um vendedor

In [11]:
df_sellers = pd.read_csv(os.path.join(DATA_DIR, 'olist_sellers_dataset.csv'))
df_sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [12]:
# Verificando a quantidade total cidades e estados onde se localizam os vendedores
print(f'Qtde. Vendedores: {df_sellers["seller_id"].nunique()}')
print(f'Qtde. Cidades: {df_sellers["seller_city"].nunique()}')
print(f'Qtde. Estado: {df_sellers["seller_state"].nunique()}')

Qtde. Vendedores: 3095
Qtde. Cidades: 611
Qtde. Estado: 23


Esta tabela apresenta dados sobre 3.095 vendedores distribuídos em 23 Estados e em um total de 611 cidades

## 1.2. Estruturando a Analytic Base Table (ABT) na visão de safras

Vamos construir a ABT utilizando apenas 3 safras: '2018-01-01', '2018-02-01' e '2018-03-01'

Para cada safra será utilizado um histórico de 18 meses, sendo os primeiros 12 meses para construir as variáveis explicativas e os últimos 6 meses para construir a variável resposta, a saber: se o vendedor realizou ou não pelo menos uma venda nos 6 meses que se seguem após a data de referência da safra.

In [25]:
df_abt_safras = pd.DataFrame()

for safra in ['2018-01-01', '2018-02-01', '2018-03-01']: # definição de 3 datas de referências para safras
  data_ref_safra = pd.to_datetime(safra).date() # transformando a string em objeto de data
  data_inf_inclusiva = data_ref_safra - dateutil.relativedelta.relativedelta(months=12) # definindo o limite inferior da safra
  data_sup_exclusiva = data_ref_safra + dateutil.relativedelta.relativedelta(months=6) # definindo o limite superior da safra

  df_historico_abt = ( # definindo dataframe com a segmentação total do período da safra
        df_order_items
        .merge(df_orders, on='order_id', how='inner') # inner join da tabela de itens com a tabela de pedidos
        .query('order_status == "delivered"') # filtrando status de pedidos entregues
        .query(f'order_approved_at >= "{data_inf_inclusiva}" & order_approved_at < "{data_sup_exclusiva}"') #filtrando os periodos totais de tempo
        .merge(df_sellers, on='seller_id', how='left') # left join do dataframe filtrado com a tabela de vendedores
    )

  df_features = ( # definindo as variáveis explicativas
        df_historico_abt
        .query(f'order_approved_at < "{data_ref_safra}"') # selecionando apenas o período de tempo aterior à data de referência
        .groupby('seller_id') # agrupando a tabela por vendedor
        .agg(uf                 = ('seller_state', 'first'), # definindo a UF do vendedor
             tot_orders_12m     = ('order_id', 'nunique'), # contabilizando o número de pedidos distintos
             tot_items_12m      = ('product_id', 'count'), # contando a quantidade de produtos vendidos
             tot_items_dist_12m = ('product_id', 'nunique'), # contabilizando a quentidade de produtos distintos comercializados
             receita_12m        = ('price', 'sum'), # contabilizando o somatório da receita
             data_ult_vnd       = ('order_approved_at', 'max')) # encontrando a data da última venda realizada no período
        .reset_index() # redefinindo o índice do dataframe
        .assign(data_ref_safra = pd.to_datetime(f'{data_ref_safra}')) # incluindo a coluna com a data de referência
        .assign(recencia = lambda df: (df['data_ref_safra'] - df['data_ult_vnd']).dt.days) # incluindo a coluna com o cálculo da recência
    )

  df_target = ( # definindo a variável resposta
        df_historico_abt
        .query(f'order_approved_at >= "{data_ref_safra}" & order_approved_at < "{data_sup_exclusiva}"') # filtrando a data igual e posterior à data de referência da safra
        .filter(['seller_id']) # selecionando apenas a coluna do vendedor
        .drop_duplicates() # removendo registros duplicados
    )

  df_abt = ( # estruturando a analytic base table
        df_features
        .merge(df_target, how='left', on='seller_id', indicator=True) # left join das variáveis explicativas com a variável resposta, com a adição de um inicador
        .assign(nao_revendeu_next_6m = lambda df: np.where(df['_merge'] == "left_only", 1, 0)) # utilizando o indicador para crirar uma flag com a informação da realização de vendas 6 meses após a data de referência
        .filter(['data_ref_safra', 'seller_id', 'uf', 'tot_orders_12m', 'tot_items_12m', 'tot_items_dist_12m', 'receita_12m', 'recencia', 'nao_revendeu_next_6m']) # filtrando e ordenando as colunas para o dataframe final
    )



df_abt_safras = pd.concat([df_abt_safras, df_abt]) # concatenando todas as safras criadas



In [26]:
df_abt_safras.head()

,data_ref_safra,seller_id,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,recencia,nao_revendeu_next_6m
0,2018-03-01,0015a82c2db000af6aaaf3ae2ecb0532,SP,3,3,1,2685.00,133,1
1,2018-03-01,001cca7ae9ae17fb1caed9dfb1094831,ES,178,209,9,21621.13,8,0
2,2018-03-01,002100f778ceb8431b7a1020ff7ab48f,SP,44,48,20,1029.20,4,0
3,2018-03-01,003554e2dce176b5555353e4f3555ac8,GO,1,1,1,120.00,75,1
4,2018-03-01,004c9cd9d87a3c30c522c48c4fc07416,SP,124,132,72,15104.92,12,0
